**Challenge Data Science - Genomawork**

*Nombre: Víctor Estrada*

El cliente Stark Industries se ha planteado el objetivo de mejorar sus contrataciones. Dentro de sus principales problemas, este identifica la rotación de personal, es decir, los colaboradores que abandonan la compañia. Para ello, se ha recolectado datos de esta empresa, para poder analizar tal fenómeno.

*   **Objetivo principal** : 
    * Hacer uso de técnicas de ciencias de datos para resolver problemas de la vida.

* **Objectivos segundarios**: 
    * Manejo de datos (leer archivos .csv, filtrarlos, etc).
    * Enfoque científico (planteamiento de hipótesis y validación o refutación de datos).
    * Uso de estadística en problemas reales.
    * Uso de herramienta de visualización de datos.
    * Técnicas de elección y entrenamiento de modelos simples de machine learning.
    * Capacidad de sintetizar los principales resultados.


---





# **Glosario de la data**

* Stag        : Experiencia (tiempo).
* Event       : Deserción de trabajo (1: Abandona, 0: Se queda).
*    Gender      : Feminino(f), Másculino(m).
*    Age         : Edad del empleado (años).
*    Industry    : Industria del empleado.
*    Profession  : Profesión del empleado.
*    Traffic     : Si el empleado vino por recomendación, paginas, etc.
*    Coach       : Presencia de un entrenador en el periodo de prueba (No,Yes,My head).
*    Head gender : Sexo del supervisor.
*    greywage    : White(salario mínimo) /Gray (pequeña cantidad por encima del mínimo).
*    Way         : Tranporte para ir a trabajar.
*    Parametros de medición de salud: extraversion, selfcontrol, etc.
---

# Librerias importadas


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# I. Carga y visualización de los datos


---
Primero que todo se importa la data (ya sea desde el computador, o en este caso, *google drive*; para luego hacer lectura de esta y visualizarla.

In [ ]:
# Just to read from google colab
#from google.colab import files

#uploaded = files.upload()

In [ ]:
# Lectura de data (archivo .csv) y visualización

file = pd.read_csv('../input/employee-turnover/turnover.csv',encoding='ISO-8859-1')
df = pd.DataFrame(file)
df.head()

# II. Análisis estadístico

**Información general**

---



In [ ]:
#Informacion principal;
df.info()

* Con la función ***.info()*** se puede obtener información general acercas de los *features* (columnas), tamaño de la muestra 1129x16,datos nulos (en este caso, no existen datos nulos), y además, el tipo de variable que representa, en este caso, se pueden encontrar: *float64, int64 y object*, lo que se traduce a que existen variables númericas y categóricas.

In [ ]:
#Información acerca de datos nulos
df.isnull().any()

* ***isnull().any()*** Nos ayuda a corrobar la verificación de datos nulos.

**Muestro de varibles categóricas/númericas**

---



In [ ]:
# Identificamos las variables categoricas

df_objects = df.select_dtypes(include=['object']).copy() # La guardamos como variable para su posterior uso.
print(df_objects.columns.values,'\n')

In [ ]:
# Luego vemos los valores únicos de tales variables

for i in df_objects.columns.values:
    print(i,':', df[i].unique(),'\n')

El objetivo de analizar los valores únicos es intentar reducir las categorias para un mejor modelado.

In [ ]:
# Identificamos las variables numéricas

df_numerical = df.select_dtypes(exclude=['object']).copy()
print(df_numerical.columns.values,'\n')

**Estadísticas**

---



In [ ]:
# Rotación de empleados

event = df['event'].value_counts()
porcentaje = [round(i*100/event.sum(),1) for i in df['event'].value_counts()]
print(event,'\n')
print('Trabajadores que renuncian : %',porcentaje[0])
print('Trabajadores que se quendan : %',porcentaje[1])

Se observa que de los datos observados existe una mayoria de personas que renuncian a sus trabajos, siendo este del % 50.6.

In [ ]:
# Promedios de valos númericos en función de la variable 'event'
df.groupby('event').mean()

**Observaciones generales**
(Repecto a la tabla anterior)
* Las personas con menor tiempo en sus trabajos tienden a irse de ellos.
* Los trabajadores más extrovertidos, independientes, novatos y con menos autocntrol tienden a renuciar.
* La edad igual parece ser influente en la estadía de trabajo, mostrándose que las personas de mayor edad tienden a la estabilidad.








**Visualización de datos**

---



A continuacíon se hace una visualización de las variables categoricas en función de la variable objetivo 'event'

In [ ]:
rows    = 3
columns = 3
c       = 1 # Inicializar plot counter

# Histograma categorical variables
fig = plt.figure(figsize=(20,20))
for i in df_objects.columns.values:

    ax = plt.subplot(rows,columns,c)
    pd.crosstab(df[i],df.event).plot(kind='bar',ax=ax)
    plt.title('Frecuencia de rotación por {}'.format(i))
    plt.ylabel('Frecuencia de rotación')
    plt.xlabel('{}'.format(i))
    plt.tight_layout(pad=4.0)
    c = c + 1

De la gráfica anterior se puede observar lo siguiente:
* Las mújeres son más probables de abandonar su lugar de trabajo
* Es claro que la frecuencia de rotación depende en gran parte de la profesión e industria, por lo que pueden ser buenas variables para el modelo predictor.

Por último, es bueno representar las variables númericas en gráficos de histogramas.

In [ ]:
num_bins = 10
histograma = df.hist(bins=num_bins, figsize=(20,20))

# III. Aplicación a Machine Learning



**Importación de librerias utilizadas**

---



In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.feature_selection import RFE, RFECV
from sklearn import metrics, preprocessing, model_selection
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC

from sklearn.model_selection import StratifiedKFold, train_test_split,cross_val_score,ShuffleSplit
from sklearn.ensemble import RandomForestClassifier

**Conversión de variables**

---



Para poder iniciar el análisis de datos mediante machine learning, debemos tener una matriz solo de variables númericas; por lo que debe transformarse las variables categóricas en "*dummy variables*".




In [ ]:
# Agreamos 1/0 para las variables categóricas

for i in df_objects.columns.values:
    object_list = 'var'+' '+ i
    object_list = pd.get_dummies(df[i],prefix = i)
    df_1 = df.join(object_list)
    df = df_1

In [ ]:
# A continuación eliminamos las variables categóricas
df = df.drop(df_objects,axis=1)

A continuación configuramos las variables para poder ingresarlas a los modelos de predicción.

In [ ]:
df_1 = df.drop('event', axis=1)  # Se elimina la variable objetivo ('event')
X = df_1.values                  # Se define la matriz X
Y = df['event'].values           # Se define el vector Y

**Normalización de variables mediante**


---


Se normalizan los valores mediante *Mean Normalization* para evitar diferencias de ordenes de magnitud.

In [ ]:
scaler = preprocessing.StandardScaler()
X2 = scaler.fit_transform(X)   ### Con este me quedo (MEAN NORMALIZATION)

**Recursive Feature Elimination (RFE)**

---



Se utiliza este método para escoger la cantidad de features más relevantes en el modelo predictivo.

In [ ]:
# PRIORIZANDO ACCURACY

rfc = RandomForestClassifier()
rfecv = RFECV(estimator= rfc, step= 1, cv=StratifiedKFold(10), scoring='accuracy')
rfecv.fit(X2,Y)


In [ ]:
print('Número óptimo de features: {}'.format(rfecv.n_features_))

plt.figure(figsize=(16, 9))
plt.title('Recursive Feature Elimination with Cross-Validation', fontsize=18, fontweight='bold', pad=20)
plt.xlabel('Number of features selected', fontsize=14, labelpad=20)
plt.ylabel('% Correct Classification', fontsize=14, labelpad=20)
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='#303F9F', linewidth=3)

plt.show()

Ahora seleccionamos y eliminamos las variables menos importantes.

In [ ]:
df_2 = df_1 # Se crea una variable auxiliar
df_2.drop(df_2.columns[np.where(rfecv.support_ == False)[0]], axis=1, inplace=True)

Visualización de variables más importantes.

In [ ]:
dset = pd.DataFrame()
dset['attr'] = df_2.columns
dset['importance'] = rfecv.estimator_.feature_importances_

dset = dset.sort_values(by='importance', ascending=False)


plt.figure(figsize=(16, 14))
plt.barh(y=dset['attr'], width=dset['importance'], color='#1976D2')
plt.title('RFECV - Feature importantes', fontsize=20, fontweight='bold', pad=20)
plt.xlabel('Importancia', fontsize=14, labelpad=20)
plt.show()

**Aplicación a modelos predictivos**

---



Se utilizaron 3 modelos de predicción: Logistic Regression, Random Forest y Support Vector Machine (SVM).

Primero que todo, se debe dividir los datos para alcanzar la máxima precisión.

In [ ]:
test_size = 0.3
random_state = 42

X_train, X_test, Y_train, Y_test = train_test_split(X2, Y, test_size= test_size, random_state= random_state)

print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', Y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)

Logistic Regression


In [ ]:
model_logreg = LogisticRegression(max_iter=100, C= 10)
model_logreg.fit(X_train, Y_train)

print('Precisión Logistic Regression: {:.3f}'.format(accuracy_score(Y_test, model_logreg.predict(X_test))))

Random Forest

In [ ]:
model_rf = RandomForestClassifier(n_estimators= 100, random_state=10, max_depth=13)
model_rf.fit(X_train,Y_train)

print('Precisión Random Forest: {:.3f}'.format(accuracy_score(Y_test, model_rf.predict(X_test))))

SVM

In [ ]:
model_svc = SVC(C=1)
model_svc.fit(X_train,Y_train)

print('Precisión Support vector machine: {:.3f}'.format(accuracy_score(Y_test, model_svc.predict(X_test))))

Luego de calcular la precisión de los modelos, se hace uso de *Cross Validation* para poder generalizar el modelo y confirmar que está bien adaptado.

In [ ]:
kfold = model_selection.KFold(n_splits=20)
modelCV = RandomForestClassifier()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, Y_train, cv=kfold, scoring=scoring)
print("Precisión promedio 10-fold cross validation: %.3f" % (results.mean()))

Como la precisión promedio resultó ser bastante cercana a la predicción del modelo, se puede decir que este está bien adaptado.

**Precision/Recall/F1 score**

---



Estas variables nos ayudan a corrobar la calidad de las predicciones de los modelos establecidos.




Logistic Regression

In [ ]:
print(classification_report(Y_test, model_logreg.predict(X_test)))

Random Forest

In [ ]:
print(classification_report(Y_test, model_rf.predict(X_test)))

SVM

In [ ]:
print(classification_report(Y_test, model_svc.predict(X_test)))

De los modelos estudiados, Random Forest es el que tiene un mayor precision, lo que indica que al momento de predecir la rotación de un trabajador, este realmente abandona su trabajo. Por otro lado Loggistic regression presentó el menor valor. Lo mismo pasa con la variable Recall, que indica cuando un trabajador abandona, cuan a menudo el modelo lo predice correctamente.

**Curva ROC**

---

Esta curva muestra la tasa de verdaderos positivos en función de falsos positivos para distintos umbrales de clasificación.



In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

logit_roc_auc = roc_auc_score(Y_test, model_logreg.predict(X_test))
fpr, tpr, thresholds = roc_curve(Y_test, model_logreg.predict_proba(X_test)[:,1])

rf_roc_auc = roc_auc_score(Y_test, model_rf.predict(X_test))
rf_fpr, rf_tpr, rf_thresholds = roc_curve(Y_test, model_rf.predict_proba(X_test)[:,1])


plt.figure()
plt.figure(figsize=(10, 10))
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot(rf_fpr, rf_tpr, label='Random Forest (area = %0.2f)' % rf_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tasa False Positive')
plt.ylabel('Tasa True Positive')
plt.title('ROC')
plt.legend(loc="lower right")
plt.savefig('ROC')

En la gráfica reciente se muestran dos modelos de predicción, los cuales están alejados de la curva roja (representa una curva pura aleatoria). Mientrás más alejados de esta, mayor precision y recall presentará el modelo.

#IV. Evaluación de un algoritmo de aprendizaje






---
**Learning Curves**

In [ ]:
##### EVALUATING A LEARNING ALGORITHM
## VALIDATION CURVES -> En función de algún "hyperparameters"
from sklearn.model_selection import validation_curve, learning_curve
from sklearn.naive_bayes import GaussianNB

def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


fig, axes = plt.subplots(3, 2, figsize=(10, 15))

title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(estimator, title, X2, Y, axes=axes[:, 0], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = SVC(gamma=0.001)
plot_learning_curve(estimator, title, X2, Y, axes=axes[:, 1], ylim=(0.0, 1.01),
                    cv=cv, n_jobs=4)

plt.show()



# Observaciones







---
* El análisis estadístico como los histogramas, nos brindan importantes observaciones a analizar a prior al modelo de predicción; dándonos una idea de que variables "podrian" tener mayor impacto en la rotación de trabajadores. Para este caso se observa que las variables, tales como, **gender, industry y profession**, tienen patrones muy marcados en la rotación. Además, los histogramas nos dan información acerca de que tan sesgada es la data recolectada, lo cual no siempre es beneficioso para el modelo de predicción.

* Con respecto a Machine learning los modelos arrojaron los siguientes resultados: Logistic regression 63.7%, Random Forest 69% y SVM 67%, con Random Forest como el modelo que más se ajusta la data. La elección de estos fue debido a su bajo coste computacional (tiempo de ejecución) y además porque soy ampliamente utilizados. Si bien soy diferentes entre si, se infiere que pudieron adaptarse de manera simil a los datos entregados. Cabe destacar, que existen otros modelos más precisos y complejos, como Neural Network, pero computacionalmente es más costoso. 

  Volviendo a los resultados arrojados por los modelos de predicción, se observa que son bastante bajos (entre 60-70%), por lo que se debe analizar el desempeño de estos con lo siguiente:

* Evaluar tu algoritmo de aprendizaje
    * Iniciando con la evaluación de tu hipótesis; que tu modelo tenga bajo error, no significa que es necesariamente una buen hipótesis (ejemplo: overfitting). Para ello se utiliza la separación de data (la cual se utilizó en este análisis).
    * En segunda instancia, aumentar el grado de tu selección de modelo y escoger la que tenga menor error.

* Análisis de Bias vs Variance
    * Para minimizar el error de un modelo (ya sea, under/over fitting), se diagnóstica como varía el error de tu modelo en función de parametros, tales como, grado, regularización y cantidad de ejemplos.
    * Si necesito ajustar la varianza, puedo agregar más datos o incrementar el parametro de regularización. Por otro lado, para arreglar el bias, es posible obtener más características.

  Con lo nombrado anteriormente, es posible disminuir el error y por ende aumentar la calidad del modelo. Lo ideal es encontrar un equilibrio entre bias y variance.  Si con estos análisis de modelos, no se logra un aumento significativo, cabe la opción es que los modelos seleccionados no se ajustan bien a la data (recordar que en esta existen variables númericas y categóricas) o tambíen cabe la posibilidad que los datos recolectados no tengan un correlación con la tasa de rotación.

* Ahora, para evaluar el desempeño de los modelos predictivos se utilizan los parámetros Precision/Recall y F1 score. De los resultados entregados se observa que Random Forest obtuvo los mayores resultados (73%/65%  Precision/Recall, respectivamente. Y 69% F1 score), y Logistic regression lo más bajos (68%, 57% y 64%).

  Cabe recordar que los parámetros Precision/Recall funcionan como un trade-off (como se muestra en el gráfico de ROC). Por lo que si quiero mejorar la precision del modelo (en este caso, predecir un empleado con riesgo de rotación) debo sacrificar el recall (a modo general). Esto significa que se debe restringir las predicciones positivas a aquellas con mayor certeza en el modelo, lo que se traduce en predecir menos resultados positivos (con esto se refiere a que, si el modelo predice una cierta cantidad de rotaciones, lo más probable es que sea conrrectas). Lo que además significa, que al tener un menor recall tendré menos aciertos de la realidad (no podré identificar a las personas con alto riesgo de rotación antes de contratarla).

* Por último se encontraron que las variables más relacionadas con la rotación son: Los parametros de medición de salud (anxiety, extraversion, selfcontrol, novator, independ), age, stage, industry retail y profession_HR, los cuales tiene un rango de importancia entre 2 y 14%.

# Conclusiones




---

El objetivo principal fue estudiar la tasas de rotación mediante Machine learning dada una cierta base de datos con datos números y no númericos. Para ello se utilizaron distintos modelos de predicción los cuales dieron como resultado una exactitud entre 64-70%, lo cual parece bastante bajo; una de las posibilidades es que los modelos no se ajustan bien a este tipo de datos (datos cuantitativos y cualitativos) o bien los datos no tienen correlación alguna con la rotación de trabajadores. Una observación importante a destacar es el sesgo de los datos; lo más óptimo es distribuir de manera equitativa, a modo de ejemplo, se tiene la profesión HR, la cual tiene la mayoria de los datos.

Para poder mejorar la predicción de los modelos se debe tener en consideración lo anteriormente mencionado, y además, lo ideal es obtener datos más influyentes en la toma decisión de rotación de los empleados; a modo de sugerencia se pueden examinar las siguientes característics: Salario (bajo-medio-alto), Promoción (aumento de sueldo/cargo) y lejanía (distancia hogar-trabajo).